## 两周内周KDJ金叉，期间日KDJ金叉且无死叉

In [52]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

current_date = datetime.now() + timedelta(days=1)
one_week_ago = current_date - timedelta(days=7)  # 计算一周之前的日期

# 计算KDJ指标
def calculate_kdj(df, n=9):
    """计算 KDJ 指标"""
    low_min = df["low"].rolling(window=n).min()
    high_max = df["high"].rolling(window=n).max()
    rsv = 100 * (df["close"] - low_min) / (high_max - low_min)
    df["K"] = rsv.ewm(com=2).mean()  # K线
    df["D"] = df["K"].ewm(com=2).mean()  # D线
    df["J"] = 3 * df["K"] - 2 * df["D"]  # J线
    return df

# 计算EMA
def calculate_ema(data, span):
    """计算EMA"""
    return data.ewm(span=span, adjust=False).mean()

# 计算MACD
def calculate_macd(df, short=12, long=26, signal=9):
    """计算MACD指标"""
    df["EMA_short"] = calculate_ema(df["close"], short)
    df["EMA_long"] = calculate_ema(df["close"], long)
    df["DIF"] = df["EMA_short"] - df["EMA_long"]
    df["DEA"] = calculate_ema(df["DIF"], signal)
    df["MACD_hist"] = 2 * (df["DIF"] - df["DEA"])
    df["Golden_Cross"] = (df["DIF"].shift(1) <= df["DEA"].shift(1)) & (
        df["DIF"] > df["DEA"]
    )
    return df

# 按周重新采样，重新计算周K线数据的KDJ
def calculate_weekly_kdj(df):
    """计算周K线的KDJ指标"""
    df["date"] = pd.to_datetime(df["date"])  # 确保 date 列是 datetime 格式
    df = df.set_index("date")  # 把 date 设为索引
    
    weekly_data = (
        df.resample("W-FRI")  # 取当周五的数据
        .agg({"open": "first", "high": "max", "low": "min", "close": "last"})
        .dropna()
    )

    # 获取当前日期
    today = pd.Timestamp.today().normalize()  # 只取日期部分，去掉时间
    last_weekly_index = weekly_data.index[-1]  # 取最后一个周五索引

    # 如果今天不是周五，修改最后一行索引为今天
    if today != last_weekly_index:
        weekly_data = weekly_data.rename(index={last_weekly_index: today})

    return calculate_kdj(weekly_data)


def find_kdj_golden_cross(df):
    """查找 KDJ 金叉和死叉，并判断当日股票涨跌"""
    df["Golden_Cross"] = (df["K"] > df["D"]) & (df["K"].shift(1) <= df["D"].shift(1))
    df["Dead_Cross"] = (df["K"] < df["D"]) & (df["K"].shift(1) >= df["D"].shift(1))

    # 判断股票涨跌：今日收盘价高于昨日收盘价则为上涨（Up），否则为下跌（Down）
    df["Price_Change"] = df["close"].diff()  # 计算收盘价变化
    df["Trend"] = df["Price_Change"].apply(lambda x: "Up" if x > 0 else "Down")

    return df


# 查找最近金叉时段 且 今日股票是涨的
def get_golden_to_dead_cross_periods(df):
    """从金叉开始到死叉结束提取日期段"""
    periods = []
    start_date = None  # 用于记录金叉的开始日期

    for i in range(len(df)):
        if df["Golden_Cross"].iloc[i]:  # 如果是金叉
            if start_date is None:  # 开始记录金叉日期
                start_date = df.index[i]
        elif df["Dead_Cross"].iloc[i]:  # 如果是死叉
            if start_date is not None:  # 如果已记录金叉
                end_date = df.index[i]
                # 不再保存金叉-死叉日期段
                # periods.append((start_date, end_date))  # 保存当前金叉-死叉日期段
                start_date = None  # 重置金叉开始日期

    # 如果循环结束时还有未匹配的金叉（没有死叉）
    if start_date is not None:
        # 检查是否小于14天
        is_within_seven_days = ((df.index[-1] - start_date)).days < 14

        if is_within_seven_days:
            periods.append((start_date, df.index[-1]))  # 配对到最后一行的日期

    return periods  # 返回金叉日期段


def filter_golden_cross_without_dead_cross(data, periods, current_date):
    # 确保索引是 DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        data = data.copy()
        data["date"] = pd.to_datetime(data["date"])  # 确保日期格式正确
        data = data.set_index("date")

    # 提取金叉日期
    golden_cross_dates = data[data["Golden_Cross"]].index

    # 检查是否在周 KDJ 的金叉至死叉时间段内，并验证无死叉条件
    filtered_dates = []
    for date in golden_cross_dates:
        # 检查金叉日期是否在周 KDJ 的金叉至死叉时间段内
        in_period = any(start <= date <= end for start, end in periods)

        if in_period:
            # 检查从金叉日期到 current_date 是否没有死叉
            no_dead_cross = all(
                not data.loc[d, "Dead_Cross"]
                for d in data.loc[date:current_date].index
                if d in data.index
            )
            if no_dead_cross:
                filtered_dates.append(date)

    # 判断今日股票涨跌的
    if data["Trend"].iloc[-1] == "Down":
        return []

    return filtered_dates


# 查找符合的股票
def get_recent_golden_cross_dates(stock_code):
    data = download_with_retry(stock_code)

    if data.empty:  # 检查是否下载失败
        print(f"Failed to download data for {stock_code}. Skipping...")
        return []

    # 换成ak了，不需要去掉第二层的ticker
    # data.columns = data.columns.droplevel(1)  # 去掉第二层的 ticker，变为单层索引

    data = calculate_kdj(data)  # 计算日 KDJ
    weekly_data = calculate_weekly_kdj(data)  # 计算周 KDJ
    weekly_data = find_kdj_golden_cross(weekly_data)  # 查找周 K线金叉
    data = find_kdj_golden_cross(data)  # 查找日线 KDJ 金叉和死叉
    periods = get_golden_to_dead_cross_periods(
        weekly_data
    )  # 提取周 KDJ 金叉到死叉的日期段

    # 获取符合条件的金叉日期
    filtered_golden_cross_dates = filter_golden_cross_without_dead_cross(
        data, periods, current_date
    )

    # 将 filtered_golden_cross_dates 转换为 pandas 的 DatetimeIndex 以便筛选
    filtered_golden_cross_dates = pd.to_datetime(filtered_golden_cross_dates)

    # 筛选出最近一周内的日期
    recent_dates = [
        date.strftime("%Y-%m-%d")
        for date in filtered_golden_cross_dates
        if one_week_ago <= date <= current_date
    ]

    return recent_dates


# 生成股票代码 名称 词典
def generate_stock_dict(filename):
    stock_dict = {}

    with open(filename, "r", encoding="utf-8") as file:
        # 读取文件内容
        content = file.read()

        # 按照股票名和股票代码的格式分割
        stock_info = content.split(")")  # 按照')'分割

        for info in stock_info:
            if "(" in info:  # 如果包含'('，说明这是一个有效的股票信息
                code_name = info.split("(")  # 通过'('分割代码和名称
                if len(code_name) == 2:
                    code = code_name[1]
                    name = code_name[0].strip()
                    stock_dict[code] = name

    return stock_dict

# 带有重试机制的 yfinance 数据下载函数。
def download_with_retry(stock_code, max_retries=2, retry_delay=3):
    end_date = current_date
    for attempt in range(max_retries):
        try:
            df = ak.stock_zh_a_cdr_daily(
                symbol=stock_code,
                start_date="2024-01-01",
                end_date=end_date.strftime("%Y-%m-%d")
            )
            if not df.empty:  # 检查数据是否下载成功
                return df
            else:
                raise ValueError("Empty DataFrame returned.")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {stock_code}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # 等待后重试
                end_date = end_date - timedelta(1)
            else:
                print(f"Max retries reached for {stock_code}. Skipping...")
    return pd.DataFrame()  # 如果失败，返回空 DataFrame

# 输出最终excel文件
def output_excel(all_golden_cross_dates, output_file):
    filename = "stock_name.txt"  # 替换新文件路径
    stock_dict = generate_stock_dict(filename)
    num = 0
    data = []
    # 遍历所有股票金叉数据
    for stock_code, dates in all_golden_cross_dates.items():
        if dates:  # 如果有金叉日期
            num += 1
            cur_stock_code = str(stock_code[-6:])  # 去掉市场后缀，保留6位股票代码
            stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
            data.append(
                {
                    "股票代码": cur_stock_code,
                    "股票名称": stock_name,
                    "一周内金叉日期": dates[0],  # 最近一周金叉日期
                }
            )

    df = pd.DataFrame(data)
    summary_row = pd.DataFrame(
        {
            "股票代码": [f"总计金叉股票个数：{num}"],
            "股票名称": [""],
            "一周内金叉日期": [""],
        }
    )  # 添加总计行到 DataFrame 的顶部

    df = pd.concat([summary_row, df], ignore_index=True)

    # output_file = "E:\GRADUATE\project\stock_info\golden_cross_summary1.xlsx"
    df.to_excel(output_file, index=False, sheet_name="金叉统计")
    print(f"Excel 文件已生成: {output_file}")


def process_stock(stock_code):
    time.sleep(random.uniform(0.1, 0.2))  # 随机延时
    try:
        recent_golden_cross_dates = get_recent_golden_cross_dates(stock_code)
        return stock_code, recent_golden_cross_dates
    except Exception as e:
        print(f"Error processing {stock_code}: {e}")
        return stock_code, []


# 替换原来的循环处理部分
def process_stocks(stock_list):
    all_golden_cross_dates = {}
    # 创建线程池，max_workers 可以根据需要调整
    with ThreadPoolExecutor(max_workers=10) as executor:
        # 提交所有任务
        future_to_stock = {
            executor.submit(process_stock, stock): stock for stock in stock_list
        }

        # 获取结果
        for future in as_completed(future_to_stock):
            stock_code, dates = future.result()
            all_golden_cross_dates[stock_code] = dates

    return all_golden_cross_dates

In [53]:
file_name = "stock_code.txt"
with open(file_name, "r") as file:
    stock_list = [line.strip() for line in file if line.strip()]

# 限制股票数量
stock_list = stock_list[:150]

all_golden_cross_dates = process_stocks(stock_list)

In [54]:
filename = "stock_name.txt"  # 替换新文件路径
stock_dict = generate_stock_dict(filename)

# 输出所有股票最近一周的金叉日期
num = 0
for stock_code, dates in all_golden_cross_dates.items():
    if dates:
        num += 1
        cur_stock_code = str(stock_code[-6:])  # 取出六位股票代码
        try:
            stock_name = stock_dict[cur_stock_code]  # 得股票具体名称
        except Exception as e:
            stock_name = "未知"
        print(
            f"股票代码: {cur_stock_code}, 股票名称: {stock_name} 一周内金叉日期: {dates[0]}"
        )

print(f"总计金叉股票个数：{num}")

股票代码: 600030, 股票名称: 中信证券 一周内金叉日期: 2025-02-21
股票代码: 600110, 股票名称: 诺德股份 一周内金叉日期: 2025-02-20
股票代码: 600171, 股票名称: 上海贝岭 一周内金叉日期: 2025-02-21
股票代码: 600187, 股票名称: 国中水务 一周内金叉日期: 2025-02-19
总计金叉股票个数：4


In [55]:
try:
    folder = Path("output")
    folder.mkdir()
except Exception as e:
    pass

In [56]:
output_file = ".\output\golden_cross_summary.xlsx"
output_excel(all_golden_cross_dates, output_file)

Excel 文件已生成: .\output\golden_cross_summary.xlsx


In [57]:
# 读取 Excel
file_path = ".\output\golden_cross_summary.xlsx"
df = pd.read_excel(file_path, usecols=[0])  # 只读取第一列

# 解决中文乱码
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

# 设置每张图最多显示的行数
rows_per_image = 50

# 按每 50 行划分数据并生成图片
for i in range(0, len(df), rows_per_image):
    subset = df.iloc[i : i + rows_per_image]  # 取出 50 行数据
    fig, ax = plt.subplots(figsize=(5, len(subset) * 0.4))  # 调整图片大小
    ax.axis("tight")
    ax.axis("off")
    ax.table(
        cellText=subset.values, colLabels=subset.columns, cellLoc="center", loc="center"
    )

    # 生成文件名，如 output_0_0.png, output_0_1.png
    output_path = f".\output\output_0_{i // rows_per_image}.png"
    plt.savefig(output_path, bbox_inches="tight", dpi=300)

    # 关闭图像，释放内存
    plt.close(fig)

print("所有图片已生成！")

所有图片已生成！


In [8]:
import zipfile
import os


def zipdir(path, ziph):
    # 循环遍历文件夹中的所有文件和子文件夹
    for root, dirs, files in os.walk(path):
        for file in files:
            # 将每个文件添加到zip文件中
            ziph.write(os.path.join(root, file))


def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, "w") as zipObj:
        # 添加文件夹及其内容到zip文件中
        zipdir(folder_path, zipObj)


zip_folder(".\output", "output.zip")

# 清理旧文件
folder_path = ".\output"
try:
    shutil.rmtree(folder_path)
except Exception as e:
    pass

In [41]:
stock_code = "sh600446"
data = download_with_retry(stock_code)
data = calculate_kdj(data)  # 计算日 KDJ
weekly_data = calculate_weekly_kdj(data)  # 计算周 KDJ
weekly_data = find_kdj_golden_cross(weekly_data)  # 查找周 K线金叉
data = find_kdj_golden_cross(data)  # 查找日线 KDJ 金叉和死叉
periods = get_golden_to_dead_cross_periods(
    weekly_data
)  # 提取周 KDJ 金叉到死叉的日期段

In [43]:
periods

[(Timestamp('2025-02-14 00:00:00'), Timestamp('2025-02-23 00:00:00'))]

In [44]:
df = weekly_data
"""从金叉开始到死叉结束提取日期段"""
periods = []
start_date = None  # 用于记录金叉的开始日期

for i in range(len(df)):
    if df["Golden_Cross"].iloc[i]:  # 如果是金叉
        if start_date is None:  # 开始记录金叉日期
            start_date = df.index[i]
    elif df["Dead_Cross"].iloc[i]:  # 如果是死叉
        if start_date is not None:  # 如果已记录金叉
            end_date = df.index[i]
            # 不再保存金叉-死叉日期段
            # periods.append((start_date, end_date))  # 保存当前金叉-死叉日期段
            start_date = None  # 重置金叉开始日期

# 如果循环结束时还有未匹配的金叉（没有死叉）
if start_date is not None:
    # 检查是否小于7天
    is_within_seven_days = ((df.index[-1] - start_date)).days < 7

    if is_within_seven_days:
        periods.append((start_date, df.index[-1]))  # 配对到最后一行的日期



In [45]:
weekly_data.tail(7)

,open,high,low,close,K,D,J,Golden_Cross,Dead_Cross,Price_Change,Trend
date,,,,,,,,,,,
2025-01-10,15.02,18.22,14.80,17.60,60.325392,67.123455,46.729268,False,False,2.43,Up
2025-01-17,17.80,17.80,15.85,17.55,61.050262,65.099057,52.952671,False,False,-0.05,Down
2025-01-24,15.80,16.15,14.25,14.56,42.787716,57.661943,13.039262,False,False,-2.99,Down
2025-01-31,15.02,15.28,14.20,14.45,30.191811,48.505233,-6.435032,False,False,-0.11,Down
2025-02-07,15.00,16.50,14.90,16.03,33.134271,43.381579,12.639655,False,False,1.58,Up
2025-02-14,15.98,17.27,15.93,17.08,43.614177,43.459111,43.924309,True,False,1.05,Up
2025-02-23,17.30,18.05,16.94,17.88,56.579855,47.832693,74.074180,False,False,0.80,Up
